# CS613: Natural language and Processing
# Word Embeddings 

## What is a Word Embedding?

Word Embedding is collective name of techniques used in natual language processing to convert words or phrases from a vocabulary into vectors of real numbers. We convert words to vectors so that the machine can understand it. We want that words having similar meaning should have similar representations. Word embeddings help us to map words to a high dimensional space such that the words that are similar are closer to each other.

<img src="vector-model1.png" alt="Alt text that describes the graphic" title="Title text" width = 400 />

## How to measure similarity?

<img src="sim.png" alt="Alt text that describes the graphic" title="Title text" width = 600 />

### 1. Manhattan Distance
<img src="manhattan.png" alt="Alt text that describes the graphic" title="Title text" width = 200 />
Manhattan distance for two vectors $x$ and $y$ is defined as follows:
<img src="manhattan_formula.gif" alt="Alt text that describes the graphic" title="Title text" width = 150 />
It is based on the distance between a grid like structure and is the sum of horizontal and vertical distances between two points on a grid. It is also known as L1 norm. 

In [1]:
#example
def manhattan_distance(x,y):
    l = len(x)
    distance = 0
    for i in range(l):
        distance += abs(x[i] - y[i])
    return distance

print("Manhatttan Distance:", manhattan_distance([1,0,0], [0,2,1]))

Manhatttan Distance: 4


### 2. Euclidean Distance
<img src="euclidean_distance.png" alt="Alt text that describes the graphic" title="Title text" width = 300 />
Euclidean distance for two vectors $x$ and $y$ is defined as follows:
<img src="euclid_eqn.gif" alt="Alt text that describes the graphic" title="Title text" width = 200 />
It is the straight line distance between two points. It is also known as L2 norm.

In [2]:
#example
import math
def euclidean_distance(x,y):
    l = len(x)
    distance = 0
    for i in range(l):
        distance += (x[i] - y[i])**2
    return math.sqrt(distance)

print("Euclidean Distance:", euclidean_distance([1,0,0], [0,2,1]))

Euclidean Distance: 2.449489742783178


### 3. Cosine Similarity
<img src="cosine.png" alt="Alt text that describes the graphic" title="Title text" width = 350 />
Cosine similarity between two vectors $x$ and $y$ is defined as follows:
<img src="eqn_similarity.svg" alt="Alt text that describes the graphic" title="Title text" width = 350 />
It is the based on the angle between two vectors and does not take into account the magnitude of the two vectors. Smaller is the angle, greater is the similarity. 
We want that similar words occupy close spatial positions i.e. they have small angle between them. Mathematically, we want that the cosine of angle between such vectors should be close to 1. This also allows us to find relations such a word is opposite of another word if cosine similarity is close to -1.  Cosine similarity is thus widely used as a measure of similarity between two words. 


In [3]:
#example
import numpy as np
def cosine_similarity(x, y):
    norm_x = math.sqrt(np.sum(np.dot(x,x))) #||x||
    norm_y = math.sqrt(np.sum(np.dot(y,y))) #||y||
    if(norm_x == 0 or norm_y == 0):
        print("Error: One of the vectors is zero")
        return -100000000
    distance = np.dot(x,y) / (norm_x * norm_y) #calculating similarity
    return distance
print("Cosine Similarity:", cosine_similarity([1,0,0], [1,2,0]))

Cosine Similarity: 0.4472135954999579


## How to convert words into vectors?

There are various techniques to convert a word into vector such as  neural networks, dimensionality reduction on the word co-occurrence matrix, probabilistic models, word2vec, glove vectors, etc.

### 1. Naive method - One Hot Encoding

One hot encoding is the traditional way of representing a word as a vector. 
<img src="onehot.png" alt="Alt text that describes the graphic" title="Title text" width = 350 />
It has 1 at only one position and zeroes at all other positions. The length of the vector is equal to the size of the vocabulary. Vocabulary is the number of unique words in our corpus. We order these words. We put one at the position for a word according to the ordering. Usually the words are ordered in aphabetical order. <br>
For example: Vocabualry = \[apple, banana, orange\]. <br> Then one hot encoding are v_apple = [1,0,0], v_banana = [0,1,0], v_orange = [0,0,1]  <br>
<br>
Issues with this approach:
1. We can not infer any relationship between two words. For example words happy and joy are similar but the cosine similarity will be zero. Also the euclidean distance will be the same for any two words. 
2. The vectors are very sparse i.e. they consist of a lot of zeroes. Thus, a lot of space is wasted.

### 2. Word2Vec

Word2vec takes into account the context(i.e. the surrounding words) of the word to find the vector of a word. There are two types of word2vec algorithms: Skip-gram, Continuous Bag of Words(CBOW). 
#### a. Skip-gram
In this we try to predict the context words based on the target word.
<img src="skipgram.png" alt="Alt text that describes the graphic" title="Title text" width = 400 />
For example if the sentence is _I am in a happy mood_ and the target word is _happy_ then the content words are _I, am, in, a, mood_ assuming the window size is 9. Window size is the number of surrounding words we take as our context. The input to the model is one hot vector of _happy_ and we try to learn the one hot encodings of the context words. In the learning process we learn the vector representation of the target word from the input layer and the weights.  

Architecture of skip-gram model:<br>
a. $W_{N,M}$ is the wieght matrix that maps the input to hidden layer. <br>
b. $W'_{M,K}$ is the wieght matrix that maps the hidden layer to the output layer. <br>

At the output layer we apply softmax and thresholding to get one hot vectors. <br>
The details of the model can be found here: __[Word2Vec](https://arxiv.org/pdf/1411.2738.pdf)__<br>
<br>
In skip gram the length of the word vectors reduces from the size of vocabulary to the dimension of the the hidden layer i.e. M.

__Implementation of skip-gram__

NLTK has various corpus from the gutenberg project. 

In [4]:
import nltk
for i in (nltk.corpus.gutenberg.fileids()):
    print(i)

austen-emma.txt
austen-persuasion.txt
austen-sense.txt
bible-kjv.txt
blake-poems.txt
bryant-stories.txt
burgess-busterbrown.txt
carroll-alice.txt
chesterton-ball.txt
chesterton-brown.txt
chesterton-thursday.txt
edgeworth-parents.txt
melville-moby_dick.txt
milton-paradise.txt
shakespeare-caesar.txt
shakespeare-hamlet.txt
shakespeare-macbeth.txt
whitman-leaves.txt


Let us take the shakespeare-ceaser.txt corpus. We preprocess the corpus as follows:

In [5]:
#performing case loweringZz
corpora = nltk.corpus.gutenberg.sents('shakespeare-caesar.txt') #taking sentences of the corpus
corpus = []
for sentence in corpora:
    sent = []
    for word in sentence:
        sent.append(word.lower())
    corpus.append(sent)
print("A sample sentence:\n", corpus[50])

A sample sentence:
 ['be', 'gone', ',', 'runne', 'to', 'your', 'houses', ',', 'fall', 'vpon', 'your', 'knees', ',', 'pray', 'to', 'the', 'gods', 'to', 'intermit', 'the', 'plague', 'that', 'needs', 'must', 'light', 'on', 'this', 'ingratitude']


In [6]:
#stopwords
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
print(stops)

{'because', 'same', "you'll", 'o', 'yourselves', 'through', 'aren', 'i', 'am', 'been', 'no', 'your', 'with', 'itself', 'a', 'doing', 'ain', 'those', 'down', 'other', 'each', 'will', 'than', 'at', 'between', 'have', 'hasn', 'his', 'myself', 'during', 'be', 'by', "should've", 'before', 'needn', 'he', 's', 'has', 'wasn', 're', 'and', 'him', 'didn', 'below', "aren't", 'being', "doesn't", 'until', "shouldn't", 'weren', 'are', 'this', 'from', 'nor', "shan't", "it's", 'very', 'm', 'them', 'do', 'just', 'over', 'she', 'their', 'hadn', "needn't", 'whom', "she's", 'does', 'all', 'was', 'haven', "weren't", "you're", 'as', 'few', 'so', 'where', 'mightn', 'there', 'is', 'these', 'more', 'in', "isn't", "haven't", 'd', 'some', 'mustn', 'above', 'himself', 'further', "that'll", "don't", 'why', 'up', 'that', 've', 'we', 'only', "wasn't", 'our', 'too', 'which', 'any', 'doesn', 'were', 'under', 'the', 'ours', 't', 'of', 'on', "wouldn't", 'most', 'yourself', 'don', 'should', 'y', 'or', 'shouldn', 'out', '

In [10]:
#removing stopwords
corpus2 = [[word for word in sent if not word in stops] for sent in corpus]
print(corpus2[50])

['gone', ',', 'runne', 'houses', ',', 'fall', 'vpon', 'knees', ',', 'pray', 'gods', 'intermit', 'plague', 'needs', 'must', 'light', 'ingratitude']


In [11]:
#punctuations
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
#removing punctuations
corpus3 = [[word for word in sent if not word in string.punctuation] for sent in corpus2]
print(corpus3[50])

['gone', 'runne', 'houses', 'fall', 'vpon', 'knees', 'pray', 'gods', 'intermit', 'plague', 'needs', 'must', 'light', 'ingratitude']


In [14]:
#making vocabulary
flatten = lambda l: [item for sublist in l for item in sublist]
vocabulary = list(set(flatten(corpus3)))
vocabulary.append('<unk>')
print(len(set(flatten(corpus))), len(vocabulary))

3032 2917


In [15]:
#creating ordering of words using word dictionary and index dictionary
d_word = {'<unk>' : 0} 

for word in vocabulary:
    d_word[word] = d_word.get(word, 0) + len(d_word)

d_ind = {v:k for k,v in d_word.items()} 

In [16]:
#defining window size
win_size = 3 #3 surrounding words on both sides. 
act_win_size = 7 #Actual window size as defined above=7. 
#<dummy> variable if it is a start or end of sentence and there are not 3 surrounding words 
windows = flatten([list(nltk.ngrams(['<dummy>'] * win_size + c + ['<dummy>'] * win_size, act_win_size)) for c in corpus])
print(windows[50])
print(windows[0])

('what', ',', 'know', 'you', 'not', '(', 'being')
('<dummy>', '<dummy>', '<dummy>', '[', 'the', 'tragedie', 'of')


In [17]:
# getting training data i.e. target, context pairs
train_data = []

for window in windows:
    for i in range(act_win_size):
        if i == win_size or window[i] == '<dummy>': 
            continue
        train_data.append((window[win_size], window[i]))

In [18]:
def check_word(word):
    return Variable(LongTensor([d_word[word]]) if d_word.get(word) is not None else LongTensor([d_word["<unk>"]]))

In [19]:
import torch

gpu_true = torch.cuda.is_available() #checking if GPU is available
torch.cuda.set_device(0)

#declaring tensors
FloatTensor = torch.cuda.FloatTensor if gpu_true else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if gpu_true else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if gpu_true else torch.ByteTensor
print(gpu_true)

True


In [20]:
#preparing training data 
X_label = []
Y_label = []
from torch.autograd import Variable
for t in train_data:
    X_label.append(check_word(t[0]).view(1, -1))
    Y_label.append(check_word(t[1]).view(1, -1))
train_data = list(zip(X_label, Y_label))

In [21]:
import torch.nn as nn

class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words,target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 

In [22]:
def index_seq(seq):
    indices = list(map(lambda w: d_word[w] if d_word.get(w) is not None else d_word["<UNK>"], seq))
    return Variable(LongTensor(indices))

In [23]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [24]:
EMBEDDING_SIZE = 5
BATCH_SIZE = 256
EPOCH = 100

In [25]:
import torch.optim as optim

losses = []
model = Skipgram(len(d_word), EMBEDDING_SIZE)
if gpu_true:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [26]:
import random
import numpy as np

for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) 
        targets = torch.cat(targets) 
        vocabs = index_seq(list(vocabulary)).expand(inputs.size(0), len(vocabulary))  # B x V
        model.zero_grad()
        loss = model(inputs, targets, vocabs)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)

    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []

Epoch : 0, mean_loss : 7.10
Epoch : 10, mean_loss : 5.95
Epoch : 20, mean_loss : 5.86
Epoch : 30, mean_loss : 5.83
Epoch : 40, mean_loss : 5.81
Epoch : 50, mean_loss : 5.80
Epoch : 60, mean_loss : 5.80
Epoch : 70, mean_loss : 5.79
Epoch : 80, mean_loss : 5.79
Epoch : 90, mean_loss : 5.79


In [34]:
def word_similarity(target, vocab):
    if gpu_true:
        target_V = model.prediction(check_word(target))
    else:
        target_V = model.prediction(check_word(target))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: continue
        
        if gpu_true:
            vector = model.prediction(check_word(list(vocab)[i]))
        else:
            vector = model.prediction(check_word(list(vocab)[i]))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0] 
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10] # sort by similarity

In [51]:
test = random.choice(list(vocabulary))
test

'superstitious'

In [52]:
import torch.nn.functional as F
word_similarity(test, vocabulary)

[['piercing', 0.9944832921028137],
 ['cursed', 0.9944651126861572],
 ['humours', 0.9907364845275879],
 ['storme', 0.9884722828865051],
 ['imminent', 0.9882675409317017],
 ['darts', 0.9843094348907471],
 ['vnto', 0.9819858074188232],
 ['lusty', 0.9801998138427734],
 ['creatures', 0.9793422222137451],
 ['tinctures', 0.9793134927749634]]

#### b. CBOW (Continuous Bag Of Words)
This model is opposite of Skip-gram model. In this model, the context words are the input and we try to predict the target word. 
<img src="cbow.png" alt="Alt text that describes the graphic" title="Title text" width = 400 />

The implementation of cboe is similar. There are various optimization methods for improving the performance of cbow such as negative sampling, hierarchial softmax, etc.<br>
Gensim library has both the implementation of skip-gram as well as cbow. 

In [46]:
import gensim
cbow = gensim.models.Word2Vec(corpus3, min_count = 1, size = 100, window = 5) 
skipgram = gensim.models.Word2Vec(corpus3, min_count = 1, size = 100, window = 5, sg = 1)
word1 = random.choice(list(vocabulary))
word2 = random.choice(list(vocabulary))
print("Cosine similarity between", word1, "and", word2, "is(cbow):", cbow.wv.similarity(word1, word2)) 
print("Cosine similarity between", word1, "and", word2, "is(skipgram):", skipgram.wv.similarity(word1, word2))

Cosine similarity between farre and done is(cbow): 0.012073921747163617
Cosine similarity between farre and done is(skipgram): 0.9863659272324679


Usually it is computationally expensive to train a neural network and also the word vectors are limited to a small corpus. The general method is to load pre-calculated word vectors.

We can do a variety of tasks with word vectors such as finding analogy, removing bias such as gender bias from vectors, etc. 

Thanks for reading!!

### References

1. https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c1
2. https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed
3. https://www.kaggle.com/watts2/glove6b50dtxt
4. https://github.com/DSKSD/DeepNLP-models-Pytorch